In [1]:
import scipy.io as scipy
import numpy as np
from time import time
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from skrvm import RVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import pandas as pd

## TrainMyClassifier function

In [2]:
def TrainMyClassifier(XEstimate, YEstimate, XValidate, TrainMyClassifierParameters):
    t0 = time()
    if TrainMyClassifierParameters[1] == 'SVM':
        # ################################################
        # Train a SVM classification model
        print("Fitting the classifier to the training set")
        param_grid = TrainMyClassifierParameters[0]
        clf = SVC(probability=True, kernel='rbf', class_weight='balanced', decision_function_shape = 'ovo',**TrainMyClassifierParameters[0])
        clf = clf.fit(XEstimate, YEstimate)
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        params = clf
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores, params
    elif TrainMyClassifierParameters[1] == 'RVM':
        # #############################################################################
        # Train a RVM classification model
        print("Fitting the classifier to the training set")
        t0 = time()
        clf = RVC(kernel='rbf',n_iter=1,**TrainMyClassifierParameters[0])
        clf.fit(XEstimate, YEstimate)
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        params = clf
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores, params
    elif TrainMyClassifierParameters[1] == 'GP':
       # #############################################################################
        # Train a GP classification model
        print("Fitting the classifier to the training set")
        t0 = time()
        k_rbf = 1 * RBF(length_scale=TrainMyClassifierParameters[0]['length_scale'])
        clf = GaussianProcessClassifier(kernel = k_rbf, multi_class = 'one_vs_one')
        clf.fit(XEstimate, YEstimate)
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        params = clf
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores, params
    else:
        print("Incorrect type of algorithm, please use only one of the supported classifiers SVM, RVM, GP")

## MyCrossValidate function

In [3]:
def MyCrossValidate(XTrain,YTrain2,Nf,Algorithm):   
    pca = PCA(n_components=9)
    reduced_data = pca.fit_transform(XTrain)
    #nfold of 5
    kf = KFold(n_splits=Nf)
    kf.get_n_splits(XTrain)
    EstParameters = []
    EstConfMatrices = []
    ConfMatrix=np.zeros((5,5),dtype=int)
    YTrain = []
    i=0        
    for train_index, test_index in kf.split(XTrain):
        
        XEst1 = XTrain[train_index]
        YEst1 = YTrain2[train_index]      
        XValid = XTrain[test_index]
        YValid = YTrain2[test_index]  
        # down-sampling to 8000
        XEst = XEst1[:2000]
        XEst=np.concatenate((XEst,XEst1[4000:6000]))
        XEst=np.concatenate((XEst,XEst1[8000:10000]))
        XEst=np.concatenate((XEst,XEst1[12000:14000,]))
        
        YEst = YEst1[:2000]    
        YEst=np.concatenate((YEst,YEst1[4000:6000]))
        YEst=np.concatenate((YEst,YEst1[8000:10000]))
        YEst=np.concatenate((YEst,YEst1[12000:14000]))
        TrainMyClassifierParameters = []
        if Algorithm =="SVM":
            TrainMyClassifierParameters.append({'C': 1e5, 'gamma': 0.3 })
        elif Algorithm =="RVM":
            TrainMyClassifierParameters.append({ 'alpha' : 1e-04, 'beta' : 1e-04 })
        elif Algorithm =="GP":
            TrainMyClassifierParameters.append({'length_scale' : 10 })
        TrainMyClassifierParameters.append(Algorithm)
        
        y_pred, scores, params = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
#         print ('Support vectors')
#         print (params.n_support_)
        c_m, df, a_s  = MyConfusionMatrix(y_pred, YValid,['One','Two','Three','Four','Five'])
        
        YTrain.append(y_pred)
        EstParameters.append(params)       
        EstConfMatrices.append(df)       
        ConfMatrix=ConfMatrix+c_m  
    ConfMatrix = ConfMatrix/5   
    return YTrain,EstParameters, EstConfMatrices,ConfMatrix

## MyConfusionMatrix function

In [4]:
def MyConfusionMatrix(Y,YValidate,ClassNames):
    c_r = classification_report(YValidate, Y)
    c_m = confusion_matrix(YValidate, Y)
    c_m = np.round((c_m / c_m.astype(np.float).sum(axis=0)), 3)
    a_s = accuracy_score(YValidate, Y)
    # labels = ['One','Two','Three','Four','Five'] - This is the format of the labels
    labels = ClassNames
    df = pd.DataFrame(c_m, dtype='str', index=labels)
    df.columns = ClassNames
    return c_m, df, a_s 

## TestMyClassifier function

In [5]:
def TestMyClassifier(XTest, Parameters, EstParameters):
    Ytests=[]
    if Parameters[1] == 'SVM':
        print("Predicting class on the test set")
        for pred_svc in EstParameters:
            t0 = time()
            Ytest=pred_svc.predict(XTest)
#             Ytest=pred_svc.predict_proba(XTest)
            Ytests.append(Ytest)
            print("done in %0.3fs" % (time() - t0))
            print(Ytest.shape)
    elif Parameters[1] == 'RVM':
        print("Predicting class on the test set")
        for pred_rvc in EstParameters:
            t0 = time()
            Ytest=pred_rvc.predict(XTest)
#             Ytest=pred_svc.predict_proba(XTest)
            Ytests.append(Ytest)
            print("done in %0.3fs" % (time() - t0))
            print(Ytest)
    elif Parameters[1] == 'GP':
        print("Predicting class on the test set")
        for pred_gp in EstParameters:
            t0 = time()
            Ytest=pred_gp.predict(XTest)
#             Ytest=pred_svc.predict_proba(XTest)
            Ytests.append(Ytest)
            print("done in %0.3fs" % (time() - t0))
            print(Ytest)
    return Ytests

## Code for experiment testing

In [6]:
# ########################################################################################
# Code for experiment testing ############################################################
# ########################################################################################

mat1 = scipy.loadmat('Proj2FeatVecsSet1.mat')
mat2 = scipy.loadmat('Proj2TargetOutputsSet1.mat')
data_set =  mat1['Proj2FeatVecsSet1']
data_target = mat2['Proj2TargetOutputsSet1']

number_labels = []
for ars in data_target:
    if np.all(ars == [1, -1, -1, -1, -1]):
        ars = 1
        number_labels.append(ars)
    elif np.all(ars == [-1, 1, -1, -1, -1]):
        ars = 2
        number_labels.append(ars)
    elif np.all(ars == [-1, -1, 1, -1, -1]):
        ars = 3
        number_labels.append(ars)
    elif np.all(ars == [-1, -1, -1, 1, -1]):
        ars = 4
        number_labels.append(ars)
    elif np.all(ars == [-1, -1, -1, -1, 1]):
        ars = 5
        number_labels.append(ars)
number_labels = np.asarray(number_labels)

X_train, X_test, y_train, y_test = train_test_split(
    data_set, number_labels, test_size=0.20, stratify = data_target)#, random_state=39)
YTrain,EstParameters, EstConfMatrices,ConfMatrix = MyCrossValidate(X_train,y_train,5,'SVM')
# print EstParameters
print EstConfMatrices
print ConfMatrix
Ytest=TestMyClassifier(X_test, [{'C': 1e5, 'gamma': 0.3 }, 'SVM'], EstParameters)
# print Ytest

Fitting the classifier to the training set
done in 3.900s
Fitting the classifier to the training set
done in 3.641s
Fitting the classifier to the training set
done in 3.439s
Fitting the classifier to the training set
done in 3.725s
Fitting the classifier to the training set
done in 3.913s
[         One    Two  Three   Four   Five
One    0.989    0.0  0.006    0.0    0.0
Two    0.005   0.97    0.0  0.014    0.0
Three  0.005  0.003  0.984  0.002  0.005
Four     0.0  0.028  0.003  0.975  0.006
Five     0.0    0.0  0.008  0.008  0.989,          One    Two  Three   Four   Five
One    0.974  0.002  0.004    0.0  0.001
Two     0.01  0.978    0.0  0.013    0.0
Three  0.006    0.0  0.978    0.0  0.005
Four     0.0  0.018  0.004  0.984  0.003
Five    0.01  0.001  0.014  0.004  0.991,          One    Two  Three   Four   Five
One    0.979    0.0  0.001  0.001    0.0
Two    0.007  0.974  0.003   0.01  0.001
Three  0.009    0.0  0.983  0.001  0.009
Four   0.004  0.026  0.004   0.98  0.004
Five   0.0